In [1]:
import numpy as np
from itertools import accumulate
from math import sqrt
np.set_printoptions(precision=5, suppress=True)

In [2]:
lista=[[1,2],[4,4],[-1,3],[16,22],[-42,0],[22,-21]]

In [3]:
homo_lista=[[7,4,1],[6,4,2],[5,2,0.5],[0,0,10],[-5,-5,1]]

In [4]:
trapez1 = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1], [1,2,3], [-8,-2,1]] 
pravougaonik1 = [[- 2, - 1, 1], [2, - 1, 1], [2, 1, 1], [- 2, 1, 1], [2,1,5], [-16,-5,5]]

In [5]:
orig=[[-3,-1,1],[3,-1,1],[1,1,1],[-1,1,1]]
slike=[[-2,-1,1],[2,-1,1],[2,1,1],[-2,1,1]]

In [6]:
def afinizuj(niz):
    return list(map(lambda x : [x[0]/x[2],x[1]/x[2]], niz))

In [7]:
def homo(niz):
    return list(map(lambda x : [x[0],x[1],1], niz))

In [8]:
def normMatrix(lista):
    lista1=np.array(afinizuj(lista))
    teziste1=(list(accumulate(lista1))[-1])/(len(lista))
    lista2=np.array(lista1-teziste1)
    inteziteti=np.array(list(map(lambda x: [sqrt(x[0]*x[0]+x[1]*x[1])],lista2)))
    d=(list(accumulate(inteziteti))[-1])/(len(lista))
    translacija=np.array([[1,0,0],[0,1,0],np.append(-teziste1,1)])
    translacija=translacija.transpose()
    homotetija=np.array([[sqrt(2)/float(d),0,0],[0,sqrt(2)/float(d),0],[0,0,1]])
    transform=np.matmul(homotetija,translacija)
    nova_lista=[]
    for x in np.array(lista):
        x=x.reshape(3,1)
        x=np.matmul(transform,x)
        nova_lista.append([x[0,0],x[1,0],x[2,0]])
    return nova_lista,transform

In [9]:
def norm_matrix(matrica):
    return matrica/matrica[2,2]

In [10]:
def DLT(originali,slike):
    matrica=[]
    for x,y in zip(originali,slike):
        prvi=[0,0,0,-y[2]*x[0],-y[2]*x[1],-y[2]*x[2],y[1]*x[0],y[1]*x[1],y[1]*x[2]]
        drugi=[y[2]*x[0],y[2]*x[1],y[2]*x[2],0,0,0,-y[0]*x[0],-y[0]*x[1],-y[0]*x[2]]
        matrica.append(prvi)
        matrica.append(drugi)
    matrica=np.array(matrica)
    U,D,VT=np.linalg.svd(matrica)
    p=VT[-1]
    return norm_matrix(p.reshape(3,3))

In [11]:
def DLTwithNormalization(originali,slike):
    originali,T=normMatrix(originali)
    slike,T1=normMatrix(slike)
    p=DLT(originali,slike)
    p=np.linalg.inv(T1)@p@T
    return norm_matrix(p)

In [12]:
def naivni(originali,slike):
    if not opsti_polozaj(originali):
        return "Losi originali!"
    if not opsti_polozaj(slike):
        return "Lose slike!"
    b=originali.pop(3)
    b1=slike.pop(3)
    x=np.linalg.solve(np.asmatrix(originali).transpose(),b)
    x1=np.linalg.solve(np.asmatrix(slike).transpose(),b1)
    x=np.array(list(map(lambda z:[z],x)))
    x1=np.array(list(map(lambda z:[z],x1)))
    p1=np.column_stack(originali*x)
    p2=np.column_stack(slike*x1)
    p=p2@(np.linalg.inv(p1))
    for i in range(0,3):
        p[i,]=list(map(lambda x: 0 if abs(x)<1e-10 else x,p[i,]))
    return norm_matrix(p)

In [13]:
def opsti_polozaj(lista):
    x1, x2, x3, x4 = lista[0], lista[1], lista[2], lista[3]
    if min(np.abs([np.linalg.det([x1, x2, x3]),
                      np.linalg.det([x2, x3, x4]),
                      np.linalg.det([x3, x4, x1]),
                      np.linalg.det([x4, x1, x2])])) == 0:
        return False
    return True

In [14]:
trapez = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1]] 
pravougaonik = [[- 2, - 1, 1], [2, - 1, 1], [2, 1, 1], [- 2, 1, 1]]
print(naivni(trapez, pravougaonik))

[[ 1.   0.   0. ]
 [ 0.   1.  -0.5]
 [ 0.  -0.5  1. ]]


In [15]:
origs = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1]] 
imgs = [[- 2, - 5, 1], [2, - 5, 1], [2, 1, 1], [6, -3, 3]]   #primetite da nisu u opstem polozaju
print(naivni(origs, imgs))

Lose slike!
